# Ses Tabanlı Duygu Tanıma (Speech Emotion Recognition - SER)
Bu notebook, **CREMA-D** ve **EMO-DB** veri setlerini kullanarak ses dosyalarından duygu tanıma işlemi gerçekleştirir. RNN (LSTM) modeli kullanılır. Adımlar:
1. **Veri Artırma**: Gürültü ekleme, hız değiştirme, perde kaydırma.
2. **Öznitelik Çıkarma**: MFCC zaman serileri.
3. **Veri Ön İşleme**: Eğitim/doğrulama/test bölmesi, StandardScaler ile ölçeklendirme.
4. **Sınıf Dengesizliği (Opsiyonel)**: SMOTE ile dengeleme.
5. **Modelleme**: LSTM tabanlı RNN modeli eğitilir ve kaydedilir.
6. **Görselleştirme**: Sınıf dağılımı, kayıp eğrileri, karışıklık matrisi ve performans metrikleri.

## 1. Gerekli Kütüphaneler

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 2. Veri Yükleme ve Öznitelik Çıkarma

In [ ]:
def load_data(dataset_path, emotion_map, dataset_name, max_per_class=50, fixed_length=32000, use_augmentation=True):
    """Veri setini yükler, artırır ve MFCC özniteliklerini zaman serisi olarak çıkarır."""
    features, labels = [], []
    sample_rate = 22050 if dataset_name == 'CREMA-D' else 16000
    print(f"{dataset_name} veri seti yükleniyor... (Sample Rate: {sample_rate})")
    
    for emotion_code, emotion_name in emotion_map.items():
        path_pattern = os.path.join(dataset_path, f'*{emotion_code}*.wav')
        files = glob.glob(path_pattern)[:max_per_class]
        if not files:
            print(f"Uyarı: {emotion_code} ({emotion_name}) sınıfında dosya bulunamadı.")
            continue
        print(f"{emotion_name} sınıfı: {len(files)} dosya bulundu.")
        for file_name in files:
            try:
                y, sr = librosa.load(file_name, sr=sample_rate, res_type='kaiser_fast')
                if len(y) == 0:
                    print(f"Uyarı: {file_name} boş veya hatalı.")
                    continue
                # Ses uzunluğunu sabitle
                if len(y) > fixed_length:
                    y = y[:fixed_length]
                else:
                    y = np.pad(y, (0, max(0, fixed_length - len(y))), mode='constant')
                
                # Veri artırma
                audios = augment_audio(y, sr, fixed_length) if use_augmentation else [y]
                for audio in audios:
                    # MFCC özniteliklerini çıkar (zaman serisi)
                    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
                    features.append(mfcc.T)  # (time_steps, n_mfcc)
                    labels.append(emotion_name)
            except Exception as e:
                print(f"Hata: {file_name} işlenemedi. Hata: {str(e)}")
    
    if not features:
        print(f"Hata: {dataset_name} için hiç öznitelik çıkarılamadı.")
        return np.array([]), np.array([])
    
    # Zaman serilerini aynı uzunluğa getir
    max_time_steps = max(f.shape[0] for f in features)
    features_padded = np.array([np.pad(f, ((0, max_time_steps - f.shape[0]), (0, 0)), mode='constant') for f in features])
    labels = np.array(labels)
    print(f"{dataset_name}: Toplam {len(features_padded)} örnek, Zaman adımı: {max_time_steps}")
    return features_padded, labels

def augment_audio(y, sr, fixed_length=32000):
    """Ses verisini artırmak için gürültü, hız değiştirme ve perde kaydırma uygular."""
    augmented = []
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) != 0 else y
    if len(y) > fixed_length:
        y = y[:fixed_length]
    else:
        y = np.pad(y, (0, fixed_length - len(y)), mode='constant')
    augmented.append(y)
    
    noise = y + 0.005 * np.random.randn(len(y))
    augmented.append(noise)
    
    try:
        speed = librosa.effects.time_stretch(y, rate=1.1)
        if len(speed) > fixed_length:
            speed = speed[:fixed_length]
        else:
            speed = np.pad(speed, (0, fixed_length - len(speed)), mode='constant')
        augmented.append(speed)
    except:
        pass
    
    try:
        pitch = librosa.effects.pitch_shift(y, sr=sr, n_steps=np.random.uniform(-1, 1))
        if len(pitch) > fixed_length:
            pitch = pitch[:fixed_length]
        else:
            pitch = np.pad(pitch, (0, fixed_length - len(pitch)), mode='constant')
        augmented.append(pitch)
    except:
        pass
    return augmented

## 3. RNN Modeli

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size=13, hidden_size=64, num_layers=2, num_classes=6):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

## 4. Veri Seti Sınıfı

In [ ]:
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

## 5. Eğitim Fonksiyonu (Görselleştirme ile)

In [ ]:
def train_rnn(model, train_loader, val_loader, dataset_name, le, num_epochs=20, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    val_accuracies, val_f1s, val_precisions, val_recalls = [], [], [], []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        # Performans metrikleri
        val_acc = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        val_precision = precision_score(val_labels, val_preds, average='weighted')
        val_recall = recall_score(val_labels, val_preds, average='weighted')
        val_accuracies.append(val_acc)
        val_f1s.append(val_f1)
        val_precisions.append(val_precision)
        val_recalls.append(val_recall)

        print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}')

        # En iyi modeli kaydet
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'rnn_model_{dataset_name}.pth')
            print(f'--> Yeni en iyi model kaydedildi (Doğruluk: {best_val_acc:.4f})')

            # En iyi model için karışıklık matrisi
            cm = confusion_matrix(val_labels, val_preds)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
            plt.title(f'RNN ({dataset_name}) En İyi Model Karışıklık Matrisi (Epoch {epoch+1})')
            plt.xlabel('Tahmin Edilen')
            plt.ylabel('Gerçek')
            plt.show()

    # Eğitim ve doğrulama kayıp eğrileri
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Eğitim Kaybı')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Doğrulama Kaybı')
    plt.title(f'{dataset_name} - Eğitim ve Doğrulama Kayıp Eğrileri')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Doğrulama performans metrikleri grafiği
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Doğruluk')
    plt.plot(range(1, num_epochs + 1), val_f1s, label='F1-Skoru')
    plt.plot(range(1, num_epochs + 1), val_precisions, label='Hassasiyet')
    plt.plot(range(1, num_epochs + 1), val_recalls, label='Duyarlılık')
    plt.title(f'{dataset_name} - Doğrulama Performans Metrikleri')
    plt.xlabel('Epoch')
    plt.ylabel('Değer')
    plt.legend()
    plt.grid(True)
    plt.show()

    return train_losses, val_losses, val_accuracies, val_f1s, val_precisions, val_recalls

## 6. Veri Seti Yolları ve Duygu Haritaları

In [ ]:
BASE_PATH = './'
CREMA_D_PATH = os.path.join(BASE_PATH, 'datasets/CREMA-D')
EMODB_PATH = os.path.join(BASE_PATH, 'datasets/EMO-DB')

crema_emotion_map = {
    'HAP': 'Happy',
    'SAD': 'Sad',
    'ANG': 'Angry',
    'FEA': 'Fear',
    'DIS': 'Disgust',
    'NEU': 'Neutral'
}

emodb_emotion_map = {
    'angry': 'Angry',
    'happy': 'Happy',
    'disgusted': 'Disgust',
    'fearful': 'Fear',
    'surprised': 'Surprised',
    'sad': 'Sad',
    'neutral': 'Neutral'
}

datasets_to_process = [
    ('CREMA-D', CREMA_D_PATH, crema_emotion_map, False),
    ('CREMA-D_AUG', CREMA_D_PATH, crema_emotion_map, True),
    ('EMO-DB', EMODB_PATH, emodb_emotion_map, False)
]

## 7. Veri Ön İşleme ve Eğitim

In [ ]:
results = []

for dataset_name, dataset_path, emotion_map, use_augmentation in datasets_to_process:
    print(f"\n{'='*20} {dataset_name} VERİ SETİ İŞLENİYOR {'='*20}")
    features, labels = load_data(dataset_path, emotion_map, dataset_name, max_per_class=50, use_augmentation=use_augmentation)
    if features.size == 0:
        print(f"{dataset_name} veri seti atlanıyor çünkü hiç örnek yüklenemedi.")
        continue

    # Etiketleri sayısal formata çevir
    le = LabelEncoder()
    labels_encoded = le.fit_transform(labels)
    joblib.dump(le, f'label_encoder_{dataset_name}.pkl')

    # Sınıf dağılımını görselleştir
    unique, counts = np.unique(labels, return_counts=True)
    plt.figure(figsize=(10, 5))
    sns.barplot(x=unique, y=counts)
    plt.title(f"{dataset_name} Orijinal Sınıf Dağılımı" + (" (Artırma Sonrası)" if use_augmentation else ""))
    plt.xlabel('Duygu')
    plt.ylabel('Örnek Sayısı')
    plt.show()

    # Veri setini böl
    X_train, X_temp, y_train, y_temp = train_test_split(features, labels_encoded, test_size=0.3, random_state=42, stratify=labels_encoded)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

    # Ölçeklendirme
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(X_val.shape[0], -1)).reshape(X_val.shape)
    X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)
    joblib.dump(scaler, f'scaler_{dataset_name}.pkl')

    # SMOTE (opsiyonel)
    use_smote = False
    if use_smote:
        smote = SMOTE(random_state=42)
        X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], -1)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train_reshaped, y_train)
        X_train_balanced = X_train_balanced.reshape(-1, X_train_scaled.shape[1], X_train_scaled.shape[2])
        print(f"{dataset_name} - SMOTE sonrası eğitim seti boyutu: {X_train_balanced.shape}")
    else:
        X_train_balanced, y_train_balanced = X_train_scaled, y_train

    # Veri setlerini hazırla
    train_dataset = AudioDataset(X_train_balanced, y_train_balanced)
    val_dataset = AudioDataset(X_val_scaled, y_val)
    test_dataset = AudioDataset(X_test_scaled, y_test)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

    # RNN modelini başlat
    num_classes = len(np.unique(labels_encoded))
    model = RNNModel(input_size=X_train_scaled.shape[2], hidden_size=64, num_layers=2, num_classes=num_classes)

    # Modeli eğit
    train_losses, val_losses, val_accuracies, val_f1s, val_precisions, val_recalls = train_rnn(
        model, train_loader, val_loader, dataset_name, le, num_epochs=20
    )

    # Sonuçları kaydet
    results.append({
        'Model': ['RNN'],
        'Accuracy': [max(val_accuracies)],
        'F1-Score': [max(val_f1s)],
        'Precision': [max(val_precisions)],
        'Recall': [max(val_recalls)],
        'Dataset': [dataset_name]
    })

    # Test verilerini kaydet
    np.save(f'X_test_{dataset_name}.npy', X_test_scaled)
    np.save(f'y_test_{dataset_name}.npy', y_test)

# Tüm veri setlerinin performans karşılaştırması
if results:
    results_df = pd.concat([pd.DataFrame(r) for r in results], ignore_index=True)

    # Doğruluk Karşılaştırması
    plt.figure(figsize=(12, 7))
    sns.barplot(x='Model', y='Accuracy', hue='Dataset', data=results_df, palette='viridis')
    plt.title('RNN Model Performans Karşılaştırması (Doğruluk)')
    plt.ylabel('Doğruluk Oranı')
    plt.xlabel('Model')
    plt.legend(title='Veri Seti')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # F1-Skoru Karşılaştırması
    plt.figure(figsize=(12, 7))
    sns.barplot(x='Model', y='F1-Score', hue='Dataset', data=results_df, palette='plasma')
    plt.title('RNN Model Performans Karşılaştırması (F1-Skoru)')
    plt.ylabel('F1-Skoru (Ağırlıklı)')
    plt.xlabel('Model')
    plt.legend(title='Veri Seti')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Hassasiyet (Precision) Karşılaştırması
    plt.figure(figsize=(12, 7))
    sns.barplot(x='Model', y='Precision', hue='Dataset', data=results_df, palette='magma')
    plt.title('RNN Model Performans Karşılaştırması (Hassasiyet)')
    plt.ylabel('Hassasiyet (Ağırlıklı)')
    plt.xlabel('Model')
    plt.legend(title='Veri Seti')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Duyarlılık (Recall) Karşılaştırması
    plt.figure(figsize=(12, 7))
    sns.barplot(x='Model', y='Recall', hue='Dataset', data=results_df, palette='inferno')
    plt.title('RNN Model Performans Karşılaştırması (Duyarlılık)')
    plt.ylabel('Duyarlılık (Ağırlıklı)')
    plt.xlabel('Model')
    plt.legend(title='Veri Seti')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
else:
    print("Karşılaştırılacak sonuç bulunamadı.")
